In [1]:
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
import csv
import pandas as pd
from time import time

In [2]:
f = open("wikidata_labels.txt", "r")
reader = csv.reader(f, delimiter=' ')

In [3]:
types = pd.read_csv("org_classes.csv", names=['0'])['0'].tolist()
#types.append("https://www.wikidata.org/entity/Q43229")
types_df = pd.read_csv("wikidata_types.txt", sep=' ', names=['entity','instanceOf','type'])
#types_df.head()
df = types_df[types_df.type.isin(types)]

In [4]:
df.head()

,entity,instanceOf,type
6895,http://www.wikidata.org/entity/Q100,http://www.wikidata.org/prop/direct/P31,http://www.wikidata.org/entity/Q21518270
6897,http://www.wikidata.org/entity/Q1000,http://www.wikidata.org/prop/direct/P31,http://www.wikidata.org/entity/Q160016
6898,http://www.wikidata.org/entity/Q1000,http://www.wikidata.org/prop/direct/P31,http://www.wikidata.org/entity/Q3624078
6900,http://www.wikidata.org/entity/Q1000,http://www.wikidata.org/prop/direct/P31,http://www.wikidata.org/entity/Q7275
6921,http://www.wikidata.org/entity/Q1000003,http://www.wikidata.org/prop/direct/P31,http://www.wikidata.org/entity/Q484170


In [6]:
def get_labels(lang='en'):
    ids = []
    labels = []
    orgs = {x:1 for x in list(set(df['entity'].tolist()))}
    for index, row in enumerate(reader):
        #print(f"processing row: {index}")
        sep_idx = row[2].rfind("@")
        if lang in row[2][sep_idx+1:]:
            if row[0] in orgs:
                ids.append(row[0])
                labels.append(row[2][:sep_idx-1])
    return ids, labels

In [7]:
start = time()
ids, lbls = get_labels()
end = time()

In [8]:
end-start

300.6121242046356

In [ ]:
#subclasses_df = pd.read_csv('wikidata_subclasses.txt', sep=' ', names=['entity','subClassOf','superClass'])
#subclasses_df.head()

In [9]:
len(lbls)

555313

In [ ]:
#labels_df = pd.read_csv('wikidata_labels.txt', sep=' ', names=['entity','predicate','label'] )
#labels_df.head()
############### Not working becuase of memory limitations

In [24]:
from ast import literal_eval as make_tuple
#f.close()
f = open("hawaii.csv", "r") #output2.csv
lines = list(f.readlines())
tuples = []
for line in lines:
    clean = line.strip().replace("\"", "").replace(",","\",\"")
    clean = f'("{clean[1:-1]}")'
    tup = make_tuple(clean)
    if len(tup) > 2:
        tup = (''.join(list(tup[:-1])), tup[-1])
    tuples.append(tup)

In [25]:
tuples[:5]

[('ABergo', 'data_hawaii_gov'),
 ('Accounting Division of Department of Accounting and General Services',
  'data_hawaii_gov'),
 ('Accounting Divisions of the Department of Accounting and General Services',
  'data_hawaii_gov'),
 ('Adrian', 'data_hawaii_gov'),
 ('aegis808', 'data_hawaii_gov')]

In [11]:
results = {}
for tup in tuples:
    org = tup[0]
    if len(org) < 2:
        continue
    for entity_id, entity_label in zip(ids,lbls):
        similarity = fuzz.ratio(org, entity_id)
        #print(f"{org} vs. {entity_label}")
        if similarity >= 85:
            print("==== Bingo ====")
            if org not in dic:
                results[org] = []
            results[org].append(entity_id)
    print(f"done with {org}")

done with Body of European Regulators for Electronic Communications
done with Cedefop
done with Committee of the Regions
done with Consumers Health Agriculture and Food Executive Agency
done with Council of the European Union
done with Directorate-General for Agriculture and Rural Development
done with Directorate-General for Communication
done with Directorate-General for Communications Networks Content and Technology
done with Directorate-General for Competition
done with Directorate-General for Economic and Financial Affairs
done with Directorate-General for Education and Culture
done with Directorate-General for Employment Social Affairs and Inclusion
done with Directorate-General for Energy
done with Directorate-General for Financial Stability Financial Services and Capital Markets Union
done with Directorate-General for Health and Food Safety
done with Directorate-General for Humanitarian Aid and Civil Protection (ECHO)
done with Directorate-General for Informatics
done with Dire

In [13]:
import requests

def fire_query(entity):
    url = 'https://query.wikidata.org/sparql'
    query = "SELECT DISTINCT ?O WHERE { ?O wdt:P31/wdt:P279* wd:Q43229; rdfs:label '"+entity+"'}"
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
    return data

In [7]:
import requests
def search_wikidata(entity):
    url = "https://www.wikidata.org/w/api.php?action=wbsearchentities&search="+entity+"&language=en&format=json"
    response = requests.request("GET", url)
    try:
        res = response.json()['search'][0]
        return res['concepturi'], res['label']
    except:
        return '', ''
search_wikidata("European Central Bank")

('http://www.wikidata.org/entity/Q8901', 'European Central Bank')

In [21]:
import requests

def search_dbpedia(entity):
    url = "http://lookup.dbpedia.org/api/search/PrefixSearch"
    querystring = {"MaxHits":"5","QueryString":entity}
    headers = {
        'Accept': "application/json",
        'cache-control': "no-cache",
        'Postman-Token': "c0a7c9ad-4277-408e-b5d6-fb03f9cbd526"
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    try:
        res = response.json()['results'][0]
        return res['uri'], res['label']
    except:
        return '', ''
    
search_dbpedia("European Central Bank")

('http://dbpedia.org/resource/European_Central_Bank', 'European Central Bank')

In [28]:
results = []
for tup in tuples:
    org = tup[0]
    if len(org) < 2:
        continue
    linked_entity, label = search_wikidata(org)
    #linked_entity, label = search_dbpedia(org)
    if len(linked_entity) > 1:
        results.append([org, linked_entity, label])
    #print(f"done with {org}")

In [6]:
results

{'Body of European Regulators for Electronic Communications': 'http://www.wikidata.org/entity/Q1297311',
 'Cedefop': 'http://www.wikidata.org/entity/Q116214',
 'Committee of the Regions': 'http://www.wikidata.org/entity/Q205203',
 'Council of the European Union': 'http://www.wikidata.org/entity/Q8896',
 'Directorate-General for Agriculture and Rural Development': 'http://www.wikidata.org/entity/Q1501751',
 'Directorate-General for Communication': 'http://www.wikidata.org/entity/Q34779544',
 'Directorate-General for Competition': 'http://www.wikidata.org/entity/Q610734',
 'Directorate-General for Economic and Financial Affairs': 'http://www.wikidata.org/entity/Q5280590',
 'Directorate-General for Education and Culture': 'http://www.wikidata.org/entity/Q5280589',
 'Directorate-General for Energy': 'http://www.wikidata.org/entity/Q15811618',
 'Directorate-General for Health and Food Safety': 'http://www.wikidata.org/entity/Q596757',
 'Directorate-General for Informatics': 'http://www.wiki

In [29]:
import csv
f = open("hawaii_baseline.csv", 'w')
writer = csv.writer(f)
writer.writerows(results)
f.close()